# PSPNet + PoseNet + ACGPN

## Dataset: VITON dataset 
    This dataset contains 16,253 image pairs, further splitting into a training set of 14,221 paris and a testing set of 2,032 pairs.

## Uploading Dataset

In [1]:
!mkdir Dataset

#### Uploading dataset in (color, edge, mask, colormask)

In [2]:
!cp -r /kaggle/input/viton-dataset/ACGPN_TestData/test_color Dataset
!cp -r /kaggle/input/viton-dataset/ACGPN_TestData/test_edge Dataset
!cp -r /kaggle/input/viton-dataset/ACGPN_TestData/test_mask Dataset
!cp -r /kaggle/input/viton-dataset/ACGPN_TestData/test_colormask Dataset

#### Uploading TestData (image, pose, label)

In [3]:
#TestData
!cp -r /kaggle/input/virtual-fit-test-dataset/Dataset/test_img Dataset
!cp -r /kaggle/input/virtual-fit-test-dataset/Dataset/test_pose Dataset
!cp -r /kaggle/input/virtual-fit-test-dataset/Dataset/test_label Dataset

In [4]:
import os
print('test image    :', len(os.listdir('./Dataset/test_img')))   # test image (person with clothes)
print('test pose     :', len(os.listdir('./Dataset/test_pose')))  # pose keypoints per test image
print('test label    :', len(os.listdir('./Dataset/test_label'))) # label (dark frame) of test image (for pose-map)
print('test color    :', len(os.listdir('./Dataset/test_color')))     # color clothes  
print('test edge     :', len(os.listdir('./Dataset/test_edge')))      # edge of clothes
print('test mask     :', len(os.listdir('./Dataset/test_mask')))      # test mask        
print('test colormask:', len(os.listdir('./Dataset/test_colormask'))) # test colormask

# Semantic Segmentation using PSPNet

In [5]:
!git clone https://github.com/rkuo2000/semantic-segmentation-pytorch 
%cd semantic-segmentation-pytorch

### Importing Important Libs

In [6]:
from torchvision import transforms
from segmentation.data_loader.segmentation_dataset import SegmentationDataset
from segmentation.data_loader.transform import Rescale, ToTensor
from segmentation.trainer import Trainer
from segmentation.predict import *
from segmentation.models import all_models
from util.logger import Logger

In [7]:
import os
from torchvision import transforms

from segmentation.data_loader.segmentation_dataset import SegmentationDataset
from segmentation.data_loader.transform import Rescale, ToTensor
from segmentation.trainer import Trainer
from segmentation.predict import *
from segmentation.models import all_models
from util.logger import Logger
import matplotlib.pyplot as plt

In [8]:
model_name = "pspnet_mobilenet_v2"
device = 'cuda'
batch_size = 4
n_classes = 34 
check_point_stride = 1 # store checkpoints every 1 epoch   
image_axis_minimum_size = 200

num_epochs = 1    # 1 for 1st training
                  # n for retraining
                  # 0 for detect-only
pretrained = False# True  for num_epochs=1 without logger.load_model below
                  # False for num_epochs=n with    logger.load_model below
                  # False for detect-only  with    logger.load_model below
fixed_feature = False

logger = Logger(model_name=model_name, data_name='example')

In [9]:
train_images = '/kaggle/input/viton-dataset/ACGPN_TrainData/train_img'
train_labled = '/kaggle/input/viton-dataset/ACGPN_TrainData/train_label'
test_images  = '/kaggle/input/viton-dataset/ACGPN_TestData/test_img'
test_labeled = '/kaggle/input/viton-dataset/ACGPN_TestData/test_label'

In [ ]:
compose = transforms.Compose([Rescale(image_axis_minimum_size),ToTensor()])

train_datasets = SegmentationDataset(train_images, train_labled, n_classes, compose)
train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=True, drop_last=True)

test_datasets = SegmentationDataset(test_images, test_labeled, n_classes, compose)
test_loader = torch.utils.data.DataLoader(test_datasets, batch_size=batch_size, shuffle=True, drop_last=True)

### Build Model

In [ ]:
### Model
model = all_models.model_from_name[model_name](n_classes, batch_size, 
                                               pretrained=pretrained, 
                                               fixed_feature=fixed_feature)
model.to(device)

# Optimizers
if pretrained and fixed_feature: #fine-tunning
    params_to_update = model.parameters()
    print("Params to learn:")
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t", name)
    optimizer = torch.optim.Adadelta(params_to_update)
else:
    optimizer = torch.optim.Adadelta(model.parameters())

In [ ]:
!mkdir -p runs/models/pspnet_mobilenet_v2/example
!cp  /kaggle/input/pspnet-viton-checkpoints/epoch_0 runs/models/pspnet_mobilenet_v2/example

logger.load_model(model, 'epoch_0') 

### Train Model

In [10]:
## Train
trainer = Trainer(model, optimizer, logger, num_epochs, train_loader, test_loader, check_point_epoch_stride=check_point_stride)
trainer.train()
print(os.listdir('runs/models/'+model_name+'/example'))

### Saving And Loading Model

In [11]:
## Loader
compose = transforms.Compose([Rescale(image_axis_minimum_size),ToTensor()])

In [12]:
model_name = "pspnet_mobilenet_v2"
device = 'cuda'
batch_size = 4
n_classes = 34 
check_point_stride = 1    
image_axis_minimum_size = 200

num_epochs = 0    
pretrained = False
fixed_feature = False

logger = Logger(model_name=model_name, data_name='example')

### Segmentation Model

In [13]:
### Model
model = all_models.model_from_name[model_name](n_classes, batch_size, 
                                               pretrained=pretrained, 
                                               fixed_feature=fixed_feature)
model.to(device)

In [14]:
#Saving and Loading Checkpoints
!mkdir -p runs/models/pspnet_mobilenet_v2/example
!cp /kaggle/input/pspnet-viton-checkpoints/epoch_1 runs/models/pspnet_mobilenet_v2/example


logger.load_model(model,'epoch_1')

## Detect label (semantic segmentation)

In [15]:
test_img_file   = '../Dataset/test_img/000001_0.jpg'
test_label_file = '../Dataset/test_label/000001_0.png'

### Pridicted Values

In [16]:
predict(model, test_img_file, test_label_file )

## Show image & its label (segmentation)

In [17]:
from IPython.display import Image

### Test Image 

In [18]:
Image(test_img_file)

###  Result test color label

In [19]:
Image(test_label_file)

### Result In Test Gray Label

In [20]:
Image(test_label_file.replace('.png','_gray.png'))

In [25]:
test_img_file   = '../Dataset/test_img/000020_0.jpg'
test_label_file = '../Dataset/test_label/000020_0.png'
predict(model, test_img_file, test_label_file )

In [26]:
Image(test_img_file)


In [27]:
Image(test_label_file)


In [28]:
Image(test_label_file.replace('.png','_gray.png'))

In [46]:
test_img_file   = '../Dataset/test_img/003821_0.jpg'
test_label_file = '../Dataset/test_label/003821_0.png'
predict(model, test_img_file, test_label_file )  

In [38]:
Image(test_img_file)

In [40]:
Image(test_label_file)

In [41]:
Image(test_label_file.replace('.png','_gray.png'))

In [54]:
test_img_file   = '../Dataset/test_img/003935_0.jpg'
test_label_file = '../Dataset/test_label/003935_0.png' 
predict(model, test_img_file, test_label_file )

In [55]:
Image(test_img_file)

In [56]:
Image(test_label_file)

In [58]:
Image(test_label_file.replace('.png','_gray.png'))

In [60]:
test_img_file   = '../Dataset/test_img/000010_0.jpg'
test_label_file = '../Dataset/test_label/000010_0.png'   
predict(model, test_img_file, test_label_file )

In [61]:
Image(test_img_file)

In [62]:
Image(test_label_file)

In [64]:
Image(test_label_file.replace('.png','_gray.png'))

In [67]:
test_img_file   = '../Dataset/test_img/001236_0.jpg'
test_label_file = '../Dataset/test_label/001236_0.png'
predict(model, test_img_file, test_label_file )        

In [69]:
Image(test_img_file)

In [70]:
Image(test_label_file)

In [72]:
Image(test_label_file.replace('.png','_gray.png'))

# Gray Label to overwrite color label as test_label .png

In [73]:
!mv ../Dataset/test_label/000000_0_gray.png ../Dataset/test_label/000000_0.png

In [74]:
%cd /kaggle/working

# Pose Detection: [PoseNet](https://arxiv.org/abs/1505.07427)

![](https://github.com/tensorflow/tfjs-models/raw/master/posenet/demos/camera.gif)

In [75]:
!pip install tfjs-graph-converter

### PoseNet-PyTorch

In [76]:
!git clone https://github.com/rwightman/posenet-pytorch  # fix bugs
%cd posenet-pytorch

In [124]:
file = '../Dataset/test_img/001236_0.jpg'

In [125]:
import torch
from posenet.constants import *
from posenet.decode_multi import decode_multiple_poses
from posenet.models.model_factory import load_model
from posenet.utils import *

net = load_model(101)
net = net.cuda()
output_stride = net.output_stride
scale_factor = 1.0

input_image, draw_image, output_scale = posenet.read_imgfile(file, scale_factor=scale_factor, output_stride=output_stride)

## Detect Pose keypoints

In [126]:
import torch
with torch.no_grad():
    input_image = torch.Tensor(input_image).cuda()

    heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = net(input_image)

    pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
        heatmaps_result.squeeze(0),
        offsets_result.squeeze(0),
        displacement_fwd_result.squeeze(0),
        displacement_bwd_result.squeeze(0),
        output_stride=output_stride,
        max_pose_detections=10,
        min_pose_score=0.25)

### find keypoint coordinates in poses

In [127]:
import matplotlib.pyplot as plt
# read image
image = plt.imread(file)

poses = []
# find face keypoints & detect face mask
for pi in range(len(pose_scores)):
    if pose_scores[pi] != 0.:
        print('Pose #%d, score = %f' % (pi, pose_scores[pi]))       
        keypoints = keypoint_coords.astype(np.int32) # convert float to integer
        print(keypoints[pi])
        poses.append(keypoints[pi])
print(len(poses))

### show PoseNet keypoints

In [128]:
img = plt.imread(file)
i=0
pose = poses[0]
plt.imshow(img)    
for y,x in pose:
    plt.plot(x, y, 'w.') # 'w.': color='white', marker='.'
    plt.text(x, y, str(i), color='r', fontsize=10)
    i+=1   
plt.show()

### PoseNet output
![](https://debuggercafe.com/wp-content/uploads/2020/10/keypoint_exmp.jpg)

### OpenPose COCO format
![](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/.github/media/keypoints_pose_18.png?raw=true)

## Convert Keypoints (from PoseNet to OpenPose)

In [129]:
# map rccpose-to-openpose mapping
indices = [0, (5,6), 6, 8, 10, 5, 7, 9, 12, 14, 16, 11, 13, 15, 2, 1, 4, 3]

In [131]:
# convert rcnnpose to openpose
i=0
openpose = []
for ix in indices:
    if ix==(5,6):
        openpose.append([int((pose[5][1]+pose[6][1])/2), int((pose[5][0]+pose[6][0])/2), 1])   
    else:
        openpose.append([int(pose[ix][1]),int(pose[ix][0]),1])        
    i+=1
    
print(openpose)

In [132]:
plt.imshow(img)
i=0
for x,y,z in openpose:
    plt.plot(x, y, 'w.') # 'w.': color='white', marker='.'
    plt.text(x, y, str(i), color='r', fontsize=10)
    i+=1
plt.show()

### save keypoints.json

In [133]:
import json
coords = []
for x,y,z in openpose:
    coords.append(float(x))
    coords.append(float(y))
    coords.append(float(z))

data = {"version": 1.0}
pose_dic = {}
pose_dic['pose_keypoints'] = coords
tmp = []
tmp.append(pose_dic)
data["people"]=tmp
print(data)

pose_name = '../Dataset/test_pose/000000_0_keypoints.json'
with open(pose_name,'w') as f:
     json.dump(data, f)  

## Verify _keypoints.json

In [134]:
%cd /kaggle/working

### read .json

In [135]:
# read pose
import numpy as np
import json
#pose_name = '/kaggle/input/tryon-testdata/Dataset/test_pose/000000_0_keypoints.json'
pose_name = 'Dataset/test_pose/000000_0_keypoints.json'
with open(pose_name, 'r') as f:
     pose_label = json.load(f)
     pose_data = pose_label['people'][0]['pose_keypoints']
     pose_data = np.array(pose_data)
     pose_data = pose_data.reshape((-1,3))
print(pose_data)
print(len(pose_data))

### draw keypoints 

In [136]:
# show pose keypoints on the test image
file = 'Dataset/test_img/000000_0.jpg'
img = plt.imread(file)
plt.imshow(img)
i=0
for x,y,z in pose_data: 
    plt.plot(x, y, 'w.') # 'w.': color='white', marker='.'
    plt.text(x, y, str(i), color='r', fontsize=10)
    i+=1
plt.show()
print(i)

# Try ON

In [137]:
!git clone https://github.com/rkuo2000/DeepFashion_Try_On
%cd DeepFashion_Try_On

### Download pre-trained model (checkpoint)

In [138]:
%cd ACGPN_inference

## Test TryOn model 

#### output : sample/000000_0.jpg

### Display TryOn result

In [144]:
!mkdir -p ../Data_preprocessing

In [145]:
!cp -rf /kaggle/input/viton-dataset/ACGPN_TestData/* ../Data_preprocessing

In [148]:
!ls ../Data_preprocessing

# Model Modified Outputs

In [150]:
!python test.py

In [151]:

!cp -rf /kaggle/input/acgpn-checkpoints/label2city checkpoints

## Results

In [152]:
!ls sample

#### Display sample images

In [153]:
from IPython.display import Image

In [154]:
Image('sample/000001_0.jpg')

In [155]:
Image('sample/000020_0.jpg')


In [156]:
Image('sample/003821_0.jpg') 

In [158]:
Image('sample/003935_0.jpg')

In [159]:
Image('sample/000010_0.jpg') 

In [160]:
Image('sample/001236_0.jpg')  

### Current Dir Location

In [162]:
import os
directory = os.getcwd()
print(directory)

In [169]:
dir = '/kaggle/working/DeepFashion_Try_On/ACGPN_inference/sample/'
results1 = []
for dir_path,_,filenames in os.walk(dir):
  results1.append(filenames)

In [170]:
results1 = results1[0]

In [ ]:
results1

In [171]:
!ls

In [172]:
import random


In [ ]:
# for i in range(0,1):
#     print(i)
#     random_result = random.choice(results)
#     Image('sample/'+str(random_result))

In [173]:
Image('sample/011769_0.jpg')


### Final Model

In [178]:
!git clone https://github.com/switchablenorms/DeepFashion_Try_On
%cd DeepFashion_Try_On

In [179]:
!ls

In [180]:
%cd ACGPN_inference

In [181]:
!mkdir -p ../Data_preprocessing

In [182]:
!cp -rf /kaggle/input/viton-dataset/ACGPN_TestData/* ../Data_preprocessing

In [183]:
!ls ../Data_preprocessing

### trained model (checkpoint)

In [184]:

!cp -rf /kaggle/input/acgpn-checkpoints/label2city checkpoints

### Virtual Fit

In [185]:
!python test.py

In [186]:
!ls sample

### Final Results

In [187]:
from IPython.display import Image

In [188]:
Image('sample/003935_0.jpg')  

In [189]:
Image('sample/000020_0.jpg') 

In [190]:
Image('sample/000001_0.jpg')

In [191]:
Image('sample/003821_0.jpg')

In [193]:
Image('sample/000010_0.jpg')

In [194]:
Image('sample/003935_0.jpg')

In [195]:
Image('sample/003821_0.jpg')

In [196]:
Image('sample/011769_0.jpg')

In [197]:
Image('sample/000010_0.jpg')

In [198]:
Image('sample/001236_0.jpg')

In [199]:
Image('sample/015697_0.jpg')

In [200]:
Image('sample/015869_0.jpg')

In [201]:
Image('sample/000192_0.jpg')

In [202]:
Image('sample/004107_0.jpg')

In [203]:
Image('sample/008032_0.jpg')

In [204]:
Image('sample/011954_0.jpg')

In [205]:
Image('sample/015881_0.jpg')

In [206]:
Image('sample/019486_0.jpg')

In [207]:
Image('sample/019279_0.jpg')

In [208]:
Image('sample/015141_0.jpg')

In [209]:
Image('sample/003514_0.jpg')

In [210]:
Image('sample/009454_0.jpg')

In [211]:
Image('sample/001448_0.jpg')

In [212]:
Image('sample/010984_0.jpg')